In [2]:
pip install pdfplumber

   ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
   ------- -------------------------------- 1.0/5.6 MB 13.2 MB/s eta 0:00:01
   ----------------------------------- ---- 5.0/5.6 MB 13.0 MB/s eta 0:00:01
   ---------------------------------------- 5.6/5.6 MB 12.6 MB/s  0:00:00
   ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
   ----------------------- ---------------- 2.1/3.5 MB 10.9 MB/s eta 0:00:01
   ---------------------------------------- 3.5/3.5 MB 9.4 MB/s  0:00:00
   ---------------------------------------- 0.0/7.1 MB ? eta -:--:--
   ----------- ---------------------------- 2.1/7.1 MB 11.1 MB/s eta 0:00:01
   ---------------------- ----------------- 3.9/7.1 MB 9.8 MB/s eta 0:00:01
   ------------------------------ --------- 5.5/7.1 MB 8.9 MB/s eta 0:00:01
   ---------------------------------------  7.1/7.1 MB 8.3 MB/s eta 0:00:01
   ---------------------------------------- 7.1/7.1 MB 8.1 MB/s  0:00:00
   ----------------------------------

In [5]:
pip install PyMuPDF

   ---------------------------------------- 0.0/18.4 MB ? eta -:--:--
    --------------------------------------- 0.3/18.4 MB ? eta -:--:--
   --- ------------------------------------ 1.8/18.4 MB 7.4 MB/s eta 0:00:03
   ------- -------------------------------- 3.4/18.4 MB 7.3 MB/s eta 0:00:03
   ---------- ----------------------------- 5.0/18.4 MB 7.3 MB/s eta 0:00:02
   ------------- -------------------------- 6.3/18.4 MB 7.3 MB/s eta 0:00:02
   ----------------- ---------------------- 7.9/18.4 MB 7.3 MB/s eta 0:00:02
   ------------------- -------------------- 9.2/18.4 MB 7.0 MB/s eta 0:00:02
   ----------------------- ---------------- 11.0/18.4 MB 7.3 MB/s eta 0:00:02
   -------------------------- ------------- 12.3/18.4 MB 7.3 MB/s eta 0:00:01
   ------------------------------ --------- 13.9/18.4 MB 7.3 MB/s eta 0:00:01
   ------------------------------- -------- 14.4/18.4 MB 6.9 MB/s eta 0:00:01
   -------------------------------- ------- 14.9/18.4 MB 6.3 MB/s eta 0:00:01
   -----

In [ ]:
import fitz  # PyMuPDF
import re
import json

In [2]:
import fitz  # PyMuPDF

pdf_path = r"C:\Users\daniil\Desktop\BRHIKO\pdf\HOL_BB_6_cut.pdf"
output_txt = r"C:\Users\daniil\Desktop\BRHIKO\pdf\HOL_BB_6.txt"

doc = fitz.open(pdf_path)

with open(output_txt, "w", encoding="utf-8") as f:
    for page_num in range(len(doc)):
        page = doc[page_num]
        text = page.get_text("text")  # wichtigste Methode

        f.write(f"\n\n=== Seite {page_num+1} ===\n\n")
        f.write(text)

print("PyMuPDF-Extraktion fertig:", output_txt)


PyMuPDF-Extraktion fertig: C:\Users\daniil\Desktop\BRHIKO\pdf\HOL_BB_6.txt


In [8]:
import re
import json

input_txt = r"C:\Users\daniil\Desktop\BRHIKO\pdf\HOL_BB_6.txt"
output_json = r"C:\Users\daniil\Desktop\BRHIKO\pdf\HOL_BB_6.json"

def is_title(line):
    # extrem simpel: viele Großbuchstaben + kein Section-Start
    if re.match(r"^\d{1,2}\.", line):
        return False
    upp = sum(c.isupper() for c in line)
    low = sum(c.islower() for c in line)
    return upp > low and len(line) > 3

section_re = re.compile(r"^(10|[1-9])\.\s*(.*)$")

with open(input_txt, "r", encoding="utf-8") as f:
    lines = [ln.rstrip("\n") for ln in f]

articles = []
current = None
i = 0
while i < len(lines):
    line = lines[i].strip()

    # Artikel-Titel
    if is_title(line):
        if current:
            articles.append(current)
        current = {
            "name": line,
            "header": "",
            "sections": {str(n): "" for n in range(1, 11)}
        }
        i += 1
        continue

    if not current:
        i += 1
        continue

    # Sectionstart
    m = section_re.match(line)
    if m:
        sec = m.group(1)
        rest = m.group(2).strip()

        buf = []
        if rest:
            buf.append(rest)

        j = i + 1
        while j < len(lines):
            nxt = lines[j].strip()
            # brich ab bei *einer echten neuen Section*
            if section_re.match(nxt):
                break
            # brich ab bei neuem Titel
            if is_title(nxt):
                break
            buf.append(nxt)
            j += 1

        # Sammeln
        text = " ".join(buf).strip()
        if text == "":
            text = "-"

        current["sections"][sec] = text
        i = j
        continue

    i += 1

# letztes speichern
if current:
    articles.append(current)

with open(output_json, "w", encoding="utf-8") as out:
    json.dump(articles, out, ensure_ascii=False, indent=2)

print("Minimalparser fertig.")


Minimalparser fertig.


In [5]:
import json
import re

INPUT = r"C:\Users\daniil\Desktop\BRHIKO\pdf\HOL_BB_6.txt"
OUTPUT = r"C:\Users\daniil\Desktop\BRHIKO\pdf\HOL_BB_6_prestructured.json"


# ============================================================
# 0. Silbentrennung reparieren
# ============================================================

def fix_hyphenation(lines):
    out = []
    i = 0
    while i < len(lines):
        line = lines[i]
        stripped = line.rstrip()

        # typische Trennung: endet auf "-" + nächste Zeile klein
        if stripped.endswith("-") and i + 1 < len(lines):
            nxt = lines[i + 1].lstrip()
            if nxt and nxt[0].islower():
                out.append(stripped[:-1] + nxt)
                i += 2
                continue

        out.append(line)
        i += 1
    return out


# ============================================================
# 1. Titel erkennen
# ============================================================

def is_article_title(line: str) -> bool:
    """Ein echter Titel ist ALL CAPS / überwiegend Großbuchstaben.
       Dinge wie 's. Strausberg' oder 'ssö Eberswalde' dürfen KEINE Titel sein.
    """

    if not line or len(line.strip()) < 2:
        return False

    # „s.“, „ssö“, „nö“, „sw“, etc.
    if re.match(r"^(s\.|ss[öo]\.|nö\.|no\.|so\.|sw\.)\s", line.lower()):
        return False

    # Sections sind keine Titel
    if re.match(r"^(10|[1-9])\.\s+", line):
        return False

    # Titel bestehen meist aus Großbuchstaben + evtl. Klammern oder Apostroph
    letters = re.findall(r"[A-Za-zÄÖÜäöüß']", line)
    if not letters:
        return False

    # Anteil Großbuchstaben
    upper = sum(1 for c in letters if c.isupper())
    ratio = upper / len(letters)

    # mindestens 60% Großbuchstaben + mindestens 1 Wort > 2 Zeichen
    if ratio >= 0.60 and any(len(w) > 2 for w in re.findall(r"[A-Za-zÄÖÜäöüß']+", line)):
        return True

    return False


# ============================================================
# 2. Section-Erkennung
# ============================================================

section_re = re.compile(r"^(10|[1-9])\.\s*(.*)$")


# ============================================================
# 3. Hauptparser
# ============================================================

def parse_file(path):
    with open(path, "r", encoding="utf-8") as f:
        lines = [ln.rstrip("\n") for ln in f.readlines()]

    # Silbentrennung 2x durchlaufen lassen = robust
    for _ in range(2):
        lines = fix_hyphenation(lines)

    articles = []
    current = None

    i = 0
    n = len(lines)

    while i < n:
        line = lines[i].strip()

        if not line:
            i += 1
            continue

        # -------------------------------------------
        # TITEL
        # -------------------------------------------
        if is_article_title(line):
            # alten Artikel abschließen
            if current:
                # header zusammenführen
                current["header"] = " ".join(current["header_lines"]).strip()
                del current["header_lines"]
                articles.append(current)

            # neuen Artikel beginnen
            current = {
                "name": line,
                "header": "",
                "header_lines": [],
                "sections": {str(k): "" for k in range(1, 11)}
            }

            # Header-Zeilen sammeln
            j = i + 1
            while j < n:
                nxt = lines[j].strip()

                if not nxt:
                    j += 1
                    continue

                # stop wenn neue section/titel
                if section_re.match(nxt):
                    break
                if is_article_title(nxt):
                    break

                # alles andere gehört zum Header
                current["header_lines"].append(nxt)
                j += 1

            i = j
            continue

        # -------------------------------------------
        # Falls noch kein Artikel offen → skip
        # -------------------------------------------
        if not current:
            i += 1
            continue

        # -------------------------------------------
        # SECTION
        # -------------------------------------------
        m = section_re.match(line)
        if m:
            sec = m.group(1)
            rest = m.group(2).strip()

            buf = []
            if rest:
                buf.append(rest)
                k = i + 1
            else:
                # Folgezeilen sammeln
                k = i + 1
                while k < n:
                    nxt = lines[k].strip()
                    if not nxt:
                        k += 1
                        continue

                    if section_re.match(nxt) or is_article_title(nxt):
                        break

                    buf.append(nxt)
                    k += 1

            text = " ".join(buf).strip()
            if not text:
                text = "-"

            current["sections"][sec] = text
            i = k
            continue

        # -------------------------------------------
        # Zeilen, die nicht zu Section gehören → an letzte volle Section
        # -------------------------------------------
        last_sec = None
        for s in range(10, 0, -1):
            if current["sections"][str(s)].strip() not in ("", "-"):
                last_sec = str(s)
                break

        if last_sec:
            current["sections"][last_sec] += " " + line
        else:
            current["header_lines"].append(line)

        i += 1

    # letzten Artikel hinzufügen
    if current:
        current["header"] = " ".join(current["header_lines"]).strip()
        del current["header_lines"]
        articles.append(current)

    return articles


# ============================================================
# 4. main()
# ============================================================

def main():
    articles = parse_file(INPUT)
    with open(OUTPUT, "w", encoding="utf-8") as f:
        json.dump(articles, f, ensure_ascii=False, indent=2)

    print("Fertig. Artikel:", len(articles))
    print("→", OUTPUT)


if __name__ == "__main__":
    main()


Fertig. Artikel: 1207
→ C:\Users\daniil\Desktop\BRHIKO\pdf\HOL_BB_6_prestructured.json


In [6]:
# normalize_json.py
# Nimmt das JSON vom Parser und normalisiert Texte (OCR-Fixes, Seite-Marker-Entfernung, Hyphenation-Reparatur),
# ohne die wichtigen Single-dashes (Section-Placeholder '-') zu löschen oder Hektar/Morgen-Angaben zu zerstören.

import json
import re

INPUT_JSON = r"C:\Users\daniil\Desktop\BRHIKO\pdf\HOL_BB_6_prestructured.json"
OUTPUT_JSON = r"C:\Users\daniil\Desktop\BRHIKO\pdf\HOL_BB_6_normalized.json"

# OCR-ersatzpaare (einfach)
REPLACEMENTS = {
    "1c12": "16/12",   # Beispielhafte Anpassung (nur wenn wirklich gewünscht, ggf. entfernen)
    "1c": "16",
    "1ö": "16",
    "  ,": ",",
    " ,": ",",
    "`": "",
    "·": "",
    "•": "",
    "‚": ",",
    "“": "\"",
    "”": "\"",
    "  ": " ",
}

def remove_page_markers(text):
    return re.sub(r"===\s*Seite\s*\d+\s*===", " ", text)

def fix_ocr_years(text):
    text = re.sub(r'\b1[BbIl](\d{2})\b', lambda m: "18" + m.group(1), text)
    text = re.sub(r'\b[Il](\d{3})\b', lambda m: "1" + m.group(1), text)
    text = re.sub(r'\b(\d)O(\d{2})\b', lambda m: m.group(1) + "0" + m.group(2), text)
    return text

def repair_hyphenation_in_text(text):
    # Repariere Silbentrennung über Zeilen (beachte: hier sind Einträge bereits Ein-Zeilen-Strings)
    # Beispiel: "Ablö- s~~g" -> "Ablösung" (sofern Buchstaben folgen)
    text = re.sub(r'([A-Za-zÄÖÜäöüß])-\s*([A-Za-zÄÖÜäöüß])', r'\1\2', text)
    # Entferne überflüssige Wiederholungen von ~ oder s~~g Artefakte
    text = text.replace("s~~g", "s")
    # Falls versehentlich "  -" (space dash) entstanden ist: behalten, aber trimme überflüssige spaces
    return text

def collapse_spaces_but_keep_single_dash(text):
    # wir wollen mehrere Whitespace reduzieren, BUT: token '-' should remain a standalone token if original was single '-'
    # normalize whitespace first
    text = re.sub(r'\s+', ' ', text)
    # trim
    return text.strip()

def normalize_field(text):
    if text is None:
        return text
    # keep original placeholder '-' intact
    if text.strip() == "-":
        return "-"

    t = text
    t = remove_page_markers(t)
    t = fix_ocr_years(t)
    t = repair_hyphenation_in_text(t)

    # perform simple replacements
    for a, b in REPLACEMENTS.items():
        t = t.replace(a, b)

    # collapse spaces while keeping single '-' tokens
    t = collapse_spaces_but_keep_single_dash(t)

    # final trim
    return t.strip()

def is_empty_article_keep_header(art):
    # Entscheide, ob Artikel lösbar ist. WICHTIG: Wenn es nur header gibt (z. B. "s. Germandorf"),
    # wollen wir den Artikel BEHALTEN (weil das bedeutende Verweisinfo enthält).
    name = (art.get("name") or "").strip()
    header = (art.get("header") or "").strip()
    sections = art.get("sections", {})

    # Wenn Name fehlt -> drop
    if not name:
        return True

    # Ghost names (Seiten- oder Kapitelüberschriften) droppen
    ghost_names = {"TEIL", "INHALT", "INDEX", "ANHANG", "REGISTER", "SEITE", "KAPITEL"}
    if name.upper() in ghost_names:
        return True

    # Wenn alle sections leer UND header leer -> drop
    all_sections_empty = all((sections.get(str(i), "").strip() in ("", "-")) for i in range(1, 11))
    if all_sections_empty and header == "":
        # But keep if the name itself looks like a genuine place (uppercase and vowel)
        if re.search(r"[AEIOUÄÖÜaeiouäöü]", name):
            # keep it (it may be a "s. ..." entry stored as name)
            return False
        return True

    return False

def main():
    with open(INPUT_JSON, "r", encoding="utf-8") as f:
        data = json.load(f)

    normalized = []
    for art in data:
        # normalize header + sections
        art["header"] = normalize_field(art.get("header", ""))
        for k in list(art.get("sections", {}).keys()):
            art["sections"][k] = normalize_field(art["sections"].get(k, ""))

        # remove accidental page markers left in sections (already done in normalize_field)
        # Drop only really empty / ghost articles
        if not is_empty_article_keep_header(art):
            normalized.append(art)
        else:
            # If it's borderline (name exists but header empty and all sections empty),
            # keep if name contains uppercase words typical for places (e.g., "ABDECKEREI")
            nm = art.get("name","").strip()
            if nm and re.match(r'^[A-ZÄÖÜ0-9\'\-\s\(\)]+$', nm) and len(nm) > 2:
                normalized.append(art)
            # else dropped

    # optional: de-duplicate exact name/header pairs preserving the first occurrence
    out = []
    seen = set()
    for art in normalized:
        key = (art.get("name","").strip().upper(), art.get("header","").strip())
        if key in seen:
            # attempt gentle merge if different sections present
            for prev in out:
                if prev.get("name","").strip().upper() == art.get("name","").strip().upper():
                    for s in prev["sections"]:
                        if prev["sections"][s].strip() in ("", "-") and art["sections"].get(s,"").strip() not in ("", "-"):
                            prev["sections"][s] = art["sections"][s]
                    if not prev["header"] and art["header"]:
                        prev["header"] = art["header"]
                    break
            continue
        out.append(art)
        seen.add(key)

    with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
        json.dump(out, f, ensure_ascii=False, indent=2)

    print("Normalizer: fertig ->", OUTPUT_JSON, "Artikel:", len(out))

if __name__ == "__main__":
    main()


Normalizer: fertig -> C:\Users\daniil\Desktop\BRHIKO\pdf\HOL_BB_6_normalized.json Artikel: 1199
